In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

C:\Users\z004bvhf\AppData\Local\Temp\ipykernel_19208\3402702372.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#Nacteni dat
data = pd.read_csv("datasets/iris.csv", decimal=',', sep=";")

# Bez posledniho sloupce
data = data.iloc[:, :-1].values
data

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [3]:
# Gaussova podobnost
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y)**2 / (2 * (sigma ** 2)))

In [4]:
def knn(data, source_index, vertex, k=3, sigma=1):
    similarities = []

    for target_index, row in enumerate(data):
      if source_index != target_index:
        similarity = gaussian_kernel(vertex, row, sigma)
        similarities.append((source_index, target_index, similarity))

    # Seřadíme podle podobnosti
    similarities.sort(key=lambda x: x[2], reverse=True)

    #K nejmensich
    k_nearest = similarities[:k]

    return k_nearest

In [5]:
def e_radius(data, source_index, vertex, epsilon=0.9, sigma=1):
  similarities = []

  for target_index, row in enumerate(data):
    if source_index != target_index:
      similarity = gaussian_kernel(vertex, row, sigma)
      if similarity > epsilon:
        similarities.append((source_index, target_index, similarity))

  # Seřadíme podle podobnosti
  similarities.sort(key=lambda x: x[2], reverse=True)

  # Vsechny nejmensi pokud maji vzdalenost
  k_nearest = similarities[:]

  return k_nearest

In [6]:
def knn_e_radius(data, source_index, vertex, k=3, epsilon=0.9, sigma=1):
  similarities = []
  #E_radius
  for target_index, row in enumerate(data):
    if source_index != target_index:
      similarity = gaussian_kernel(vertex, row, sigma)
      if similarity > epsilon:
        similarities.append((source_index, target_index, similarity))

  # Seřadíme podle podobnosti
  similarities.sort(key=lambda x: x[2], reverse=True)

  # KNN
  # K nejmensich
  k_nearest = similarities[:k]

  return k_nearest

In [7]:
# KNN
output_results = []
for source_index, vertex in enumerate(data):
  result_knn = knn(data, source_index, vertex, k=3, sigma=1)

  for line in result_knn:
    if (line[1], line[0],line[2]) not in output_results and (line[0], line[1],line[2]) not in output_results:
      if line[0] < line[1]:
        output_results.append(line)
      else:
        output_results.append((line[1],line[0],line[2]))

# Seřaď podle Source, Target
output_results.sort(key=lambda x: (x[0], x[1]))

results_df = pd.DataFrame(output_results, columns=["Source", "Target", "Sim"])

results_df
results_df.to_csv('knn.csv', index=False)

In [8]:
# E-radius

output_results = []
for source_index, vertex in enumerate(data):
  result_knn = e_radius(data, source_index, vertex, epsilon=0.9, sigma=1)

  for line in result_knn:
    if (line[1], line[0],line[2]) not in output_results and (line[0], line[1],line[2]) not in output_results:
      if line[0] < line[1]:
        output_results.append(line)
      else:
        output_results.append((line[1],line[0],line[2]))

# Seřaď podle Source, Target
output_results.sort(key=lambda x: (x[0], x[1]))

results_df = pd.DataFrame(output_results, columns=["Source", "Target", "Sim"])

#results_df
results_df.to_csv('e_r.csv', index=False)

In [9]:
# K_nn + E-radius
output_results = []

for source_index, vertex in enumerate(data):
  result_knn = knn_e_radius(data, source_index, vertex, k=3, epsilon=0.9, sigma=1)
  
  for line in result_knn:
    if (line[1], line[0],line[2]) not in output_results and (line[0], line[1],line[2]) not in output_results:
      if line[0] < line[1]:
        output_results.append(line)
      else:
        output_results.append((line[1],line[0],line[2]))

# Seřaď podle Source, Target
output_results.sort(key=lambda x: (x[0], x[1]))

results_df = pd.DataFrame(output_results, columns=["Source", "Target", "Sim"])

#results_df
results_df.to_csv('knn_e_radius.csv', index=False)